In [1]:
import pandas as pd

# read the data from CSV file
# data = pd.read_csv('/Users/mohamedgani/Downloads/Exhibits/combined_file.csv')
data = pd.read_csv('/Users/mohamedgani/Downloads/cluster_0_1_2.csv')
data = data.dropna()

In [2]:
data

,id,url,b title,corresponding p text,cluster,cluster_2,preprocessed p text
0,0,https://www.sec.gov/Archives/edgar/data/127501...,Design of Due Diligence Measures,The Company has established management systems...,2,0,company established management system due dili...
1,0,https://www.sec.gov/Archives/edgar/data/127501...,Due Diligence Measures Taken,The Company first identified suppliers and als...,2,1,company first identified supplier also analyze...
2,0,https://www.sec.gov/Archives/edgar/data/127501...,Due Diligence Results,"Through its efforts, the Company received info...",2,0,effort company received information 26 supplie...
3,1,https://www.sec.gov/Archives/edgar/data/320193...,Upstream Due Diligence,"Each year, we analyze incident data provided b...",2,1,year analyze incident data provided itsci rcs ...
4,1,https://www.sec.gov/Archives/edgar/data/320193...,OECD Step 5: Report on Supply Chain Due Diligence,Apple reports annually on its due diligence re...,2,2,apple report annually due diligence requiremen...
...,...,...,...,...,...,...,...
8738,11264,https://www.sec.gov/Archives/edgar/data/704532...,Due Diligence Process,Due Diligence Process The Company’s supply cha...,2,1,due diligence process company ’ supply chain r...
8739,11264,https://www.sec.gov/Archives/edgar/data/704532...,Design of Due Diligence,The Company’s due diligence measures have been...,2,0,company ’ due diligence measure designed confo...
8740,11264,https://www.sec.gov/Archives/edgar/data/704532...,Description of Due Diligence Measures,The\nCompany’s due diligence on the source and...,2,1,company ’ due diligence source chain custody c...
8741,11264,https://www.sec.gov/Archives/edgar/data/704532...,Results of Due Diligence,Identify and Assess Risk in the Supply Chain I...,2,0,identify ass risk supply chain identification ...


In [3]:
# Select the rows where cluster_2 is 0 which is results
cluster_1_0 = data[data['cluster_2'] == 1]

In [4]:
cluster_1_0

,id,url,b title,corresponding p text,cluster,cluster_2,preprocessed p text
1,0,https://www.sec.gov/Archives/edgar/data/127501...,Due Diligence Measures Taken,The Company first identified suppliers and als...,2,1,company first identified supplier also analyze...
3,1,https://www.sec.gov/Archives/edgar/data/320193...,Upstream Due Diligence,"Each year, we analyze incident data provided b...",2,1,year analyze incident data provided itsci rcs ...
6,8,https://www.sec.gov/Archives/edgar/data/183563...,Due Diligence Performed,Step 1: Establish Strong Company Management Sy...,2,1,step 1 establish strong company management sys...
8,17,https://www.sec.gov/Archives/edgar/data/142080...,Due Diligence Program,Design Framework Design Framework We designed ...,2,1,design framework design framework designed due...
9,17,https://www.sec.gov/Archives/edgar/data/142080...,Selected Elements of our Due Diligence Program,The OECD Guidance has established a five-step ...,2,1,oecd guidance established fivestep framework d...
...,...,...,...,...,...,...,...
8733,11258,https://www.sec.gov/Archives/edgar/data/109158...,Section 1: Due diligence framework,"In accordance with Rule 13p-1, we undertook du...",2,1,accordance rule 13p1 undertook due diligence e...
8734,11258,https://www.sec.gov/Archives/edgar/data/109158...,Section 2: Due diligence measures undertaken,Our due diligence efforts for 2013 included th...,2,1,due diligence effort 2013 included following f...
8735,11258,https://www.sec.gov/Archives/edgar/data/109158...,Step 4: Carry out independent third-party audi...,Step 4: Carry out independent third-party audi...,2,1,step 4 carry independent thirdparty audit smel...
8738,11264,https://www.sec.gov/Archives/edgar/data/704532...,Due Diligence Process,Due Diligence Process The Company’s supply cha...,2,1,due diligence process company ’ supply chain r...


In [5]:
def get_coherence(model, token_lists, measure='c_v'):
    """
    Get model coherence from gensim.models.coherencemodel
    :param model: Topic_Model object
    :param token_lists: token lists of docs
    :param topics: topics as top words
    :param measure: coherence metrics
    :return: coherence score
    """
    if model.method == 'LDA':
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    else:
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    return cm.get_coherence()

def get_silhouette(model):
    """
    Get silhouette score from model
    :param model: Topic_Model object
    :return: silhouette score
    """
    if model.method == 'LDA':
        return
    lbs = model.cluster_model.labels_
    vec = model.vec[model.method]
    return silhouette_score(vec, lbs)

In [6]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


###################################
#### sentence level preprocess ####
###################################

# lowercase + base filter
# some basic normalization
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()


# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """

    # some reviews are actually english but biased toward french
    return detect_language(s) in {'English', 'French','Spanish','Chinese'}


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]


# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']


# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            pass
            # do word segmentation, deprecated for inefficiency
            # w_seg = sym_spell.word_segmentation(phrase=word)
            # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
p_stemmer = PorterStemmer()


def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [p_stemmer.stem(word) for word in w_list]


# filtering out stop words
# create English stop words list

stop_words = (list(
    set(get_stop_words('en'))
    |set(get_stop_words('es'))
    |set(get_stop_words('de'))
    |set(get_stop_words('it'))
    |set(get_stop_words('ca'))
    #|set(get_stop_words('cy'))
    |set(get_stop_words('pt'))
    #|set(get_stop_words('tl'))
    |set(get_stop_words('pl'))
    #|set(get_stop_words('et'))
    |set(get_stop_words('da'))
    |set(get_stop_words('ru'))
    #|set(get_stop_words('so'))
    |set(get_stop_words('sv'))
    |set(get_stop_words('sk'))
    #|set(get_stop_words('cs'))
    |set(get_stop_words('nl'))
    #|set(get_stop_words('sl'))
    #|set(get_stop_words('no'))
    #|set(get_stop_words('zh-cn'))
))





def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]


def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s


def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    w_list = f_noun(w_list)
    w_list = f_typo(w_list)
    w_list = f_stem(w_list)
    w_list = f_stopw(w_list)

    return w_list

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
#from Autoencoder import *
#from preprocess import *
from datetime import datetime


# def preprocess(docs, samp_size=None):
#     """
#     Preprocess the data
#     """
#     if not samp_size:
#         samp_size = 100

#     print('Preprocessing raw texts ...')
#     n_docs = len(docs)
#     sentences = []  # sentence level preprocessed
#     token_lists = []  # word level preprocessed
#     idx_in = []  # index of sample selected
#     #     samp = list(range(100))
#     samp = np.random.choice(n_docs, samp_size)
#     for i, idx in enumerate(samp):
#         sentence = preprocess_sent(docs[idx])
#         token_list = preprocess_word(sentence)
#         if token_list:
#           idx_in.append(idx)
#           sentences.append(sentence)
#           token_lists.append(token_list)
#         print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')
#     print('Preprocessing raw texts. Done!')
#     return sentences, token_lists, idx_in
def preprocess(docs, samp_size=None):
    """
    Preprocess the data
    """
    if not samp_size:
        samp_size = 100
        
    docs = docs.reset_index(drop=True)     ####Added this line to resetting and sending to preprocess
    print('Preprocessing raw texts ...')
    n_docs = len(docs)
    sentences = []  # sentence level preprocessed
    token_lists = []  # word level preprocessed
    idx_in = []  # index of sample selected
    #     samp = list(range(100))
    samp = np.random.choice(n_docs, samp_size)
    for i, idx in enumerate(samp):
        if idx >= n_docs:
            continue
        sentence = preprocess_sent(docs[idx])
        token_list = preprocess_word(sentence)
        if token_list:
          idx_in.append(idx)
          sentences.append(sentence)
          token_lists.append(token_list)
        print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')
    print('Preprocessing raw texts. Done!')
    return sentences, token_lists, idx_in


In [13]:
# define model object
class Topic_Model:
    def __init__(self, k=9, method='LDA_BERT'):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
            raise Exception('Invalid method!')
        self.k = k
        self.dictionary = None
        self.corpus = None
        #         self.stopwords = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        #self.gamma = 15  # parameter for reletive importance of lda
        self.gamma = 30
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    def vectorize(self, sentences, token_lists, method=None):
        """
        Get vecotr representations from selected methods
        """
        # Default method
        if method is None:
            method = self.method

        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
            print('Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('Getting vector representations for TF-IDF. Done!')
            return vec

        elif method == 'LDA':
            print('Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)

            def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            print('Getting vector representations for LDA. Done!')
            return vec

        elif method == 'BERT':

            print('Getting vector representations for BERT ...')
            from sentence_transformers import SentenceTransformer
            model = SentenceTransformer('bert-base-nli-max-tokens')
            vec = np.array(model.encode(sentences, show_progress_bar=True))
            print('Getting vector representations for BERT. Done!')
            return vec

             
        elif method == 'LDA_BERT':
        #else:
            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]
            self.vec['LDA_BERT_FULL'] = vec_ldabert
            if not self.AE:
                self.AE = Autoencoder()
                print('Fitting Autoencoder ...')
                self.AE.fit(vec_ldabert)
                print('Fitting Autoencoder Done!')
            vec = self.AE.encoder.predict(vec_ldabert)
            return vec

    def fit(self, sentences, token_lists, method=None, m_clustering=None):
        """
        Fit the topic model for selected method given the preprocessed data
        :docs: list of documents, each doc is preprocessed as tokens
        :return:
        """
        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('Fitting LDA ...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                print('Fitting LDA Done!')
        else:
            print('Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, method)
            self.cluster_model.fit(self.vec[method])
            print('Clustering embeddings. Done!')

    def predict(self, sentences, token_lists, out_of_sample=None):
        """
        Predict topics for new_documents
        """
        # Default as False
        out_of_sample = out_of_sample is not None

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)
        else:
            corpus = self.corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)
        return lbs

In [14]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mohamedgani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mohamedgani/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=Warning)

import argparse

#def model(): #:if __name__ == '__main__':
from datetime import datetime       ### Added for error handling

    
    
method = "LDA_BERT"
#samp_size = 1000

    #samp_size = 500
ntopic = 10
    
    #parser = argparse.ArgumentParser(description='contextual_topic_identification tm_test:1.0')

    #parser.add_argument('--fpath', default='/kaggle/working/train.csv')
    #parser.add_argument('--ntopic', default=10,)
    #parser.add_argument('--method', default='TFIDF')
    #parser.add_argument('--samp_size', default=20500)
    
    #args = parser.parse_args()

data = cluster_1_0['corresponding p text'] #pd.read_csv('/kaggle/working/train.csv')
data = data.fillna('')  # only the comments has NaN's
#samp_size = len(data)
samp_size = len(data)
rws = data

In [ ]:
sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)
    # Define the topic model object
    #tm = Topic_Model(k = 10), method = TFIDF)
tm = Topic_Model(k = ntopic, method = method)
    # Fit the topic model by chosen method
tm.fit(sentences, token_lists)

Preprocessing raw texts ...
Preprocessing raw texts. Done!
Clustering embeddings ...
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for BERT ...


In [ ]:
# Run splitting the code

In [ ]:
sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)

In [ ]:
# Create a new dataframe with the output columns
output_df = pd.DataFrame({'sentences': sentences, 'token_lists': token_lists, 'idx_in': idx_in})
# Save the output dataframe to a csv file
output_df.to_csv('preprocessed_data.csv', index=False)

In [ ]:
import pandas as pd
data = pd.read_csv('/Users/mohamedgani/Downloads/preprocessed_data.csv')
data = data.dropna()

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=Warning)

import argparse
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
#def model(): #:if __name__ == '__main__':


    
    
method = "LDA_BERT"
#samp_size = 1000

    #samp_size = 500
ntopic = 10
tm = Topic_Model(k = ntopic, method = method)
    # Fit the topic model by chosen method
    
sentence_list = data['sentences'].tolist()
token_lists_list = data['token_lists'].tolist()
# for lst in data['token_lists']:
#     if not all(isinstance(item, str) for item in lst):
#         raise ValueError("Token lists should be a list of list of strings")
# tm.fit(data['sentence_list'], data['token_lists_list'])
tm.fit(sentence_list, token_lists_list)

In [ ]:
sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)
    # Define the topic model object
    #tm = Topic_Model(k = 10), method = TFIDF)
tm = Topic_Model(k = ntopic, method = method)
    # Fit the topic model by chosen method
tm.fit(sentences, token_lists)

In [ ]:
method = "LDA_BERT"

ntopic = 10

In [ ]:
data = cluster_1_0['preprocessed p text']

In [ ]:
data

In [ ]:
samp_size = len(data)
rws = data

rws

In [ ]:
tm = Topic_Model(k = ntopic, method = method)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoTokenizer, TFAutoModel


# Create a BERT transformer model and a tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

# Convert preprocessed text data to tokenized BERT inputs
input_ids = []
for text in data['preprocessed_text']:
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    input_ids.append(encoded_text)
input_ids = np.array(input_ids)

# Train LDA model on tokenized BERT inputs
vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tf = vectorizer.fit_transform(input_ids)
lda_model = LatentDirichletAllocation(n_components=9, max_iter=10, learning_method='online', verbose=1, random_state=42)
lda_model.fit(tf)

# Assign each text document in dataframe to a cluster
clusters = lda_model.transform(tf)
df['cluster'] = np.argmax(clusters, axis=1)

# Calculate top keywords for each cluster
feature_names = vectorizer.get_feature_names()
top_keywords = []
for topic_idx, topic in enumerate(lda_model.components_):
    top_keywords.append(", ".join([feature_names[i] for i in topic.argsort()[:-6:-1]]))

# Create final output dataframe
output = []
for idx, row in df.iterrows():
    output.append([idx, row['cluster'], top_keywords[row['cluster']], *clusters[idx], row['update_description']])
final_output = pd.DataFrame(output, columns=["doc_index", "Cluster_Number", "Cluster Keywords", "Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3", "Cluster 4", "Cluster 5", "Cluster 6", "Cluster 7", "Cluster 8", "Update Description"])